In [ ]:
bronze_path = "abfss://<container>@<storage-account>.dfs.core.windows.net/bikestore/bronze/"
silver_path = "abfss://<container>@<storage-account>.dfs.core.windows.net/bikestore/silver/"
gold_path   = "abfss://<container>@<storage-account>.dfs.core.windows.net/bikestore/gold/"
resource_path = "/Volumes/<CATALOG>/<SCHEMA>/bikestore_resource/origem"
resource_path_volume = '/Volumes/aulas_databricks/azure/bikestore_resource/origem/'


In [0]:
%python
silver_map = {
    #"tmp_silver_customer":      f"{silver_path}/customer/",
    "tmp_silver_orders":        f"{silver_path}/orders/",
    #"tmp_silver_product":       f"{silver_path}/product/",

}
for view_name, path in silver_map.items():
    (spark.read.format('delta')
        .load(path)
        .createOrReplaceTempView(view_name))
 


In [0]:
%python
df_sales_ny_gold = spark.sql("""
                               
select 
  shipped_date
  ,round(sum(total_sale),2) as total_sale
  --,state,status
 from tmp_silver_orders 
 where state = 'NY'
 and status = 'Delivered'
 and shipped_date is not null
 group by shipped_date
 --,state  ,status
         
                              
                              """)

# salvar em Delta na gold
df_sales_ny_gold.write\
    .mode('overwrite')\
    .format('delta')\
    .option('mergeSchema','true')\
    .save(f'{gold_path}/sales_ny')
    